# Gemini & ChatGPT

In [ ]:
print(
"""
***************************************
00_Initialize_LLM_Jerry.ipynb
2025/03/15 001 version
***************************************
"""
)

In [ ]:
# %env GOOGLE_LOGIN=TRUE
# %env REINSTALL=TRUE

In [6]:
%%bash
echo "REINSTALL: $REINSTALL"
if [ "$REINSTALL" == "TRUE" ]
then
  # requirements.txt가 존한다면
  if [ -f requirements.txt ]
  then
    pip install -r requirements.txt
  else
    pip install -r langchain_study/requirements.txt
  fi
fi

In [ ]:
import os

print(f"GOOGLE_LOGIN: {os.environ.get('GOOGLE_LOGIN')}")
if "GOOGLE_LOGIN" in os.environ and os.environ["GOOGLE_LOGIN"] == "TRUE":
  from google.colab import auth
  auth.authenticate_user()

In [ ]:
!pip list | grep langchain

In [8]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import json

# public colab인지 일반 jupyter server인지 확인
colab = True
credentials = {}
try:
    from google.colab import userdata
    api_key = userdata.get('openai-key')
except:
    colab = False

print("Colab:", colab)

if colab == False:
    print("Colab에 Secret이 없으므로, credentials.json 파일을 검색합니다.")
    print("사용할 항목에만 Secret을 입력해주세요.")
    print(f"credentials.json 예제: https://github.com/javalove93/langchain_study/raw/refs/heads/main/credentials-sample.json")
    PWD = !pwd
    print(f"  현재 디랙토리: {PWD[0]}")
    print("  lang-chain/credentials.json")
    if os.path.exists("lang-chain/credentials.json"):
        with open("lang-chain/credentials.json") as f:
            credentials = json.load(f)
    else:
        print("  failed")
        print("  현재 디랙토리: ./credentials.json")
        if os.path.exists("credentials.json"):
            with open("credentials.json") as f:
                credentials = json.load(f)
                print("  success")
        else:
            print("  failed")
            print("  ../credentials.json")
            if os.path.exists(os.path.expanduser("../credentials.json")):
                with open(os.path.expanduser("../credentials.json")) as f:
                    credentials = json.load(f)
                    print("  success")
            else:
                print("  failed")
                print("  ../../credentials.json")
                if os.path.exists(os.path.expanduser("../../credentials.json")):
                    with open(os.path.expanduser("../../credentials.json")) as f:
                        credentials = json.load(f)
                        print("  success")
                else:
                    print("  failed")
                    print("  ../../../credentials.json")
                    if os.path.exists(os.path.expanduser("../../../credentials.json")):
                        with open(os.path.expanduser("../../../credentials.json")) as f:
                            credentials = json.load(f)
                            print("  success")
                    else:
                        print("credentials.json 파일을 찾을 수 없습니다.")

# Open AI (불필요 하면 제거)
if colab:
    api_key = userdata.get('openai-key')
else:
    api_key = credentials.get('openai-key')

os.environ['OPENAI_API_KEY'] = api_key
OPENAI_API_KEY = api_key
print(f"OPENAI_API_KEY: {OPENAI_API_KEY[:5]}")

# openai_llm = ChatOpenAI(openai_api_key=API_KEY)
openai_llm = ChatOpenAI()
# print(openai_llm.invoke("Yes means Yes, then No means [ ]?"))

# Anthropic (불필요 하면 제거)
if colab:
    ANTHROPIC_API_KEY = userdata.get('anthropic-key')
else:
    ANTHROPIC_API_KEY = credentials.get('anthropic-key')

os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
print(f"ANTHROPIC_API_KEY: {ANTHROPIC_API_KEY[:5]}")

anthropic_llm = ChatAnthropic(model="claude-3-opus-20240229")
# print(anthropic_llm.invoke("Yes means Yes, then No means [ ]?"))

# Google Gemini (불필요 하면 제거)
if colab:
    GOOGLE_API_KEY = userdata.get('gemini-api-key')
else:
    GOOGLE_API_KEY = credentials.get('gemini-api-key')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print(f"GOOGLE_API_KEY: {GOOGLE_API_KEY[:5]}")

gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# setup the gemini pro
# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versioning?hl=ko#stable-versions-available.md
# text-embedding-004	2024년 5월 14일	해당 없음
# text-multilingual-embedding-002	2024년 5월 14일	해당 없음
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/multimodal-embeddings-api?hl=ko
# 멀티모달 임베딩	multimodalembedding@001
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="text-multilingual-embedding-002")
# print(gemini_llm.invoke("Yes means Yes, then No means [ ]?"))

llm = gemini_llm

from vertexai.language_models import TextEmbeddingModel
class GoogleTextCustomEmbedding:
    def __init__(self, model_name="text-multilingual-embedding-002"):
        self.text_model = TextEmbeddingModel.from_pretrained(model_name)

    def embed_documents(self, documents):
        embeds = self.text_model.get_embeddings(documents)
        return [e.values for e in embeds]
    
    def embed_query(self, query):
        return self.text_model.get_embeddings([query])[0].values

# Google Search Wrapper (불필요 하면 제거)
if colab:
    SERPER_API_KEY = userdata.get('google-serper-key')
else:
    SERPER_API_KEY = credentials.get('google-serper-key')

os.environ["SERPER_API_KEY"] = SERPER_API_KEY
print(f"SERPER_API_KEY: {SERPER_API_KEY[:5]}")

# def print_message(aiMessage, messageOnly=True):
#     print("RESPONSE: {}".format(aiMessage.content))
#     if messageOnly == False:
#         print(json.dumps(aiMessage.additional_kwargs, indent=2))
#         print(json.dumps(aiMessage.response_metadata, indent=2))
#         print(json.dumps(aiMessage.usage_metadata, indent=2))

# Lang Smith Tracing (API Key 발급받아야 하며, 불필요하면 제거)
%env LANGCHAIN_TRACING_V2=true
%env LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
if colab:
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith')
else:
    os.environ["LANGCHAIN_API_KEY"] = credentials.get('langsmith')
%env LANGCHAIN_PROJECT=langgraph-test
print(f"LANGCHAIN_API_KEY: {os.environ['LANGCHAIN_API_KEY'][:5]}")

# Google Programmable Search API
if colab:
    GOOGLE_SEARCH_API_KEY = userdata.get('google-search-api-key')
    GOOGLE_SEARCH_CSE_ID = userdata.get('google-search-cse')
else:
    GOOGLE_SEARCH_API_KEY = credentials.get('google-search-api-key')
    GOOGLE_SEARCH_CSE_ID = credentials.get('google-search-cse')
# API Key가 GenAI API Key와 출돌(권한이 다를 수 있어서 환경 변수에 저장 안함
# os.environ["GOOGLE_API_KEY"] = GOOGLE_SEARCH_API_KEY
# os.environ["GOOGLE_CSE_ID"] = GOOGLE_SEARCH_CSE_ID
print(f"GOOGLE_SEARCH_API_KEY: {GOOGLE_SEARCH_API_KEY[:5]}")
print(f"GOOGLE_SEARCH_CSE_ID: {GOOGLE_SEARCH_CSE_ID[:5]}")

# PINECONE API
if colab:
    PINECONE_API_KEY = userdata.get('pinecone.io')
else:
    PINECONE_API_KEY = credentials.get('pinecone.io')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
print(f"PINECONE_API_KEY: {PINECONE_API_KEY[:5]}")

# Kaggle 계정 초기화
if colab:
    KAGGLE_USERNAME = userdata.get('KAGGLE_USERNAME')
    KAGGLE_KEY = userdata.get('KAGGLE_KEY')
else:
    KAGGLE_USERNAME = credentials.get('KAGGLE_USERNAME')
    KAGGLE_KEY = credentials.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = KAGGLE_USERNAME
os.environ["KAGGLE_KEY"] = KAGGLE_KEY
print(f"KAGGLE_KEY: {KAGGLE_KEY[:5]}")

# https://smith.langchain.com

In [ ]:
import os

print(f"SA_KEY_ENABLE: {os.environ.get('SA_KEY_ENABLE')}")
if "SA_KEY_ENABLE" in os.environ and os.environ["SA_KEY_ENABLE"] == "TRUE":
    # Service Account Key
    if colab:
        SA_KEY = userdata.get('google-sa-json')
    else:
        SA_KEY = credentials.get('google-sa-json')

    with open("/tmp/sa_key.json", "w") as f:
        sa_key = json.loads(SA_KEY)
        f.write(json.dumps(SA_KEY))
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/tmp/sa_key.json"
    print(f"GOOGLE_APPLICATION_CREDENTIALS: /tmp/sa_key.json")